#### NCSU GIS 714: Geospatial Computation and Simulations

## Data simulation: Part 1

Contents:
* Start GRASS GIS
* Simple Deterministic Surfaces: Planes and Waves
* Stochastic Surfaces

Additional Resources:
* [GRASS GIS Overview and Manual](https://grass.osgeo.org/grass-stable/manuals/index.html)
* [GRASS book](http://www.grassbook.org/)

#### Start GRASS GIS

In [ ]:
import subprocess
import sys

# Ask GRASS GIS where its Python packages are.
# FOR WINDOWS:
# grass_call = "grass83"
# shell_setting = True
# FOR MAC/LINUX
grass_call = "grass"
shell_setting = False

sys.path.append(
    subprocess.check_output([grass_call, "--config", "python_path"], text=True, shell=shell_setting).strip()
)

# Import GRASS packages
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
gj.init("../data", "nc_spm_08_grass7", "PERMANENT")

# Make a new mapset for this assignment
gs.run_command("g.mapset", mapset="HW2_data_simulation", location="nc_spm_08_grass7", flags="c")

# Set the region to small rural area and print region info
print(gs.read_command("g.region", region="rural_1m", flags="p"))

### Simple deterministic surfaces

#### Waves
In the lecture we have shown mathematical surface with hills and valleys generated using the following function:

In [ ]:
gs.mapcalc("waves = sin(0.4*row())+0.3*cos(col())+80")

gs.run_command("r.colors", map="waves", co="plasma", flags="e")

# Create Map3D instance
img = gj.Map3D()
img.render(elevation_map="waves", color_map="waves", perspective=20, height=150)
# Display map
img.show()

#### *Question*

Using map algebra generate surface with hills, depressions or valleys that is different from the one shown in the lecture,
but is close enough to the elev_lid792_1m surface that you can visualize them together in 3D. Describe your work.

In [ ]:
#YOUR CODE AND TEXT HERE

### Stochastic methods

Investiagte impact of noise with different distributions on water flow pattern, watershed and stream 
delineation using random surfaces. Read the manual pages for the relevant commands to understand 
the methods and parameters. 

First we explore different types of random surfaces: uniform with values within the interval [-2,2],
Gaussian distribution around the mean=0 with sigma=0.3, and spatially dependent gaussian distribution with
distance 10m. Then we add these noise surfaces to our elevation surface.

##### Uniform

In [ ]:
gs.run_command("r.surf.random", out="uniform", min=-2, max=2)
print(gs.read_command("r.univar", map="uniform"))

In [ ]:
gs.run_command("r.colors", map="uniform", co="bcyr")

img = gj.Map()
img.d_rast(map="uniform")
img.d_legend(raster="uniform", at=(5,50,7,10), flags="b")
img.show()

In [ ]:
# Add uniform noise to elevation
gs.mapcalc("{c} = {a} + {b}".format(c="elev_lidnoise_uniform", a="elev_lid792_1m", b="uniform"))
gs.run_command("r.colors", map="elev_lidnoise_uniform", co="elevation")

##### Gaussian

In [ ]:
gs.run_command("r.surf.gauss", out="gauss_m0_s03", mean=0, sigma=0.3)
print(gs.read_command("r.univar", map="gauss_m0_s03"))

In [ ]:
# Render Gaussian raster
gs.run_command("r.colors", map="gauss_m0_s03", co="differences")

img = gj.Map()
img.d_rast(map="gauss_m0_s03")
img.d_legend(raster="gauss_m0_s03", at=(5,50,7,10), flags="b")
img.show()

In [ ]:
# Add gaussian noise to elevation
gs.mapcalc("{c} = {a} + {b}".format(c="elev_lidnoise_gauss", a="elev_lid792_1m", b="gauss_m0_s03"))
gs.run_command("r.colors", map="elev_lidnoise_gauss", co="elevation")

##### Spatially Dependent Gaussian Distribution

In [ ]:
gs.run_command("r.random.surface", output="gauss_spatial_d10", distance=10)
print(gs.read_command("r.univar", map="gauss_spatial_d10"))

In [ ]:
# Render Spatially-Dependent Gaussian raster
gs.run_command("r.colors", map="gauss_spatial_d10", co="differences")

img = gj.Map()
img.d_rast(map="gauss_spatial_d10")
img.show()

In [ ]:
# Add gaussian noise to elevation
gs.mapcalc("{y} = ({x} - 134.441) * (0.3/254)".format(y="gauss_surfspat", x="gauss_spatial_d10"))
gs.mapcalc("{c} = {a} + {b}".format(c="elev_lidnoise_gspat", a="elev_lid792_1m", b="gauss_surfspat"))

gs.run_command("r.colors", map="elev_lidnoise_gspat", co="elevation")

#### *Question*

Can you explain the constants used in the mapcalc command for gauss_surfspat?
Where do these constants come from and why are they needed?

->YOUR ANSWER HERE

#### Comparison

In [ ]:
# Create Interactive Map
fig = gj.InteractiveMap(width = 600, use_region=True)
# Add raster, vector and layer control to map
fig.add_raster("elev_lidnoise_uniform", opacity=1.0, title="Uniform Noise")
fig.add_raster("elev_lidnoise_gauss", opacity=1.0, title="Gaussian Noise")
fig.add_raster("elev_lidnoise_gspat", opacity=1.0, title="Spatially-Dependent Gaus. Noise")
fig.add_raster("elev_lid792_1m", opacity=1.0, title="Elevation, no add'l noise")
fig.add_layer_control(position = "bottomright")
# Display map
fig.show()

#### Flow Accumulation Comparison

In [ ]:
gs.run_command("r.watershed", elevation="elev_lid792_1m", thresh=5000, accum="accum_5K", drain="draindir_5K", basin="basin_5K", flags="a")
gs.run_command("r.watershed", elevation="elev_lidnoise_uniform", thresh=15000, accum="accum_15K_noise", drain="draindir_15K_noise", basin="basin_15K_noise", flags="a")
gs.run_command("r.watershed", elevation="elev_lidnoise_gspat", thresh=10000, accum="accum_10K_noisegspat", drain="draindir_10K_noisegspat", basin="basin_10K_noisegspat", flags="a")

#### *Question*
Visualy compare flow accumulation patterns derived from the original DEM (`elev_lid792_1m`), the DEM with Uniform noise (`elev_lidnoise_uniform`), and the DEM with spatially-dependent gaussian noise (`elev_lidnois_gspat`). What can you say about the spatial pattern of flow accumulation derived from these three DEMs?

In [ ]:
### YOUR CODE AND TEXT HERE

#### *Optional: Question*
For quantitative comparison, you can extract selected basins and compare their area using r.report, compute distances between the boundaries, extract streams using r.stream.extract and compare their characteristics using the *r.stream* add-ons. 


### Additional Information
* [Website](https://github.com/ncsu-geoforall-lab/geospatial-simulations-course)
    
* [Computing Help](https://help.ncsu.edu/)
    
* [GIST Home](https://geospatial.ncsu.edu/)
    
* [Disclaimer](https://www.ncsu.edu/policies/prr-disclaimer.php)
    
* [Accessibility](https://oit.ncsu.edu/itaccess)

* License: 2018 [CC BY-SA](https://creativecommons.org/licenses/by-sa/4.0/)
  
* [NCSU GeoForAll Lab](https://geospatial.ncsu.edu/geoforall/)